In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8933, 28, 28) (8933,)
Test set (8676, 28, 28) (8676,)


Reformat into a shape that's more adapted to the models we're going to train:

- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8933, 784) (8933, 10)
Test set (8676, 784) (8676, 10)


Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Let's run this computation and iterate:

In [7]:
num_steps = 801


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) /
            predictions.shape[0])


with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' %
                  accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' %
                  accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.296728
Training accuracy: 5.7%
Validation accuracy: 9.0%
Loss at step 100: 2.289851
Training accuracy: 71.6%
Validation accuracy: 69.5%
Loss at step 200: 1.852584
Training accuracy: 74.7%
Validation accuracy: 71.7%
Loss at step 300: 1.614621
Training accuracy: 76.1%
Validation accuracy: 72.3%
Loss at step 400: 1.453194
Training accuracy: 76.9%
Validation accuracy: 72.7%
Loss at step 500: 1.333391
Training accuracy: 77.7%
Validation accuracy: 72.9%
Loss at step 600: 1.239272
Training accuracy: 78.2%
Validation accuracy: 73.0%
Loss at step 700: 1.162593
Training accuracy: 78.9%
Validation accuracy: 73.3%
Loss at step 800: 1.098466
Training accuracy: 79.3%
Validation accuracy: 73.6%
Test accuracy: 81.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size,
                                             image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run([optimizer, loss, train_prediction],
                                        feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" %
                  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" %
                  accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.001106
Minibatch accuracy: 14.1%
Validation accuracy: 18.4%
Minibatch loss at step 500: 0.957261
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 1000: 1.223871
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 0.910966
Minibatch accuracy: 81.2%
Validation accuracy: 76.6%
Minibatch loss at step 2000: 0.832549
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 0.965528
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 0.900832
Minibatch accuracy: 79.7%
Validation accuracy: 77.6%
Test accuracy: 85.2%


Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

### more hidden

In [10]:
n_hidden = 1024
batch_size = 128

g = tf.Graph()
with g.as_default():

    # Placehodlers for training dataset minibatches
    tf_train_dataset = tf.placeholder(np.float32,
                                      shape=(batch_size,
                                             image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32,
                                     shape=(batch_size, num_labels))

    # Constants for validation and test datasets
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Define Weights and biases as variables for hidden layer
    W_0 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
    b_0 = tf.Variable(tf.zeros([n_hidden]))

    # Define Weights and biases as variables for logistic regression layer
    W_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    b_1 = tf.Variable(tf.zeros([num_labels]))

    logits_0 = tf.nn.relu(tf.matmul(tf_train_dataset, W_0) +
                          b_0)  # shape will be (batch_size, n_hidden)

    logits_1 = tf.matmul(
        logits_0, W_1) + b_1  # now the shape is (batch_size, num_labels)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                logits=logits_1))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions
    train_prediction = tf.nn.softmax(logits_1)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W_0) + b_0), W_1) +
        b_1)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W_0) + b_0), W_1) +
        b_1)

In [11]:
n_steps = 5001  # number of steps to be taken

with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(n_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Prepare minibatch data
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                     feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch  train and validation accuracy: %.1f%%, %.1f%%" %
                  (accuracy(predictions, batch_labels),
                   accuracy(valid_prediction.eval(), valid_labels)))
    
    print("Test accuracy: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 399.977112
Minibatch  train and validation accuracy: 11.7%, 26.7%
Minibatch loss at step 500: 18.999050
Minibatch  train and validation accuracy: 82.8%, 79.4%
Minibatch loss at step 1000: 12.634210
Minibatch  train and validation accuracy: 77.3%, 80.3%
Minibatch loss at step 1500: 6.636210
Minibatch  train and validation accuracy: 89.1%, 79.7%
Minibatch loss at step 2000: 3.118363
Minibatch  train and validation accuracy: 87.5%, 80.4%
Minibatch loss at step 2500: 3.528680
Minibatch  train and validation accuracy: 84.4%, 81.5%
Minibatch loss at step 3000: 3.241815
Minibatch  train and validation accuracy: 80.5%, 81.6%
Minibatch loss at step 3500: 3.647437
Minibatch  train and validation accuracy: 84.4%, 81.1%
Minibatch loss at step 4000: 3.906032
Minibatch  train and validation accuracy: 85.9%, 81.3%
Minibatch loss at step 4500: 5.535872
Minibatch  train and validation accuracy: 85.9%, 80.3%
Minibatch loss at step 5000: 4.158406
Minibatch  train and

### regularization techniques.

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import timeit

In [2]:
sanitized_pickle_file = 'notMNIST_sanitized.pickle'

with open(sanitized_pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    sanit_valid_dataset = save['valid_dataset']
    sanit_valid_labels = save['valid_labels']
    sanit_test_dataset = save['test_dataset']
    sanit_test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('sanit_Validation set', sanit_valid_dataset.shape,
          sanit_valid_labels.shape)
    print('sanit_Test set', sanit_test_dataset.shape, sanit_test_labels.shape)

Training set (200000, 28, 28) (200000,)
sanit_Validation set (8933, 28, 28) (8933,)
sanit_Test set (8676, 28, 28) (8676,)


In [3]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(sanit_valid_dataset, sanit_valid_labels)
test_dataset, test_labels = reformat(sanit_test_dataset, sanit_test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('sanit_Validation set', valid_dataset.shape, valid_labels.shape)
print('sanit_Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
sanit_Validation set (8933, 784) (8933, 10)
sanit_Test set (8676, 784) (8676, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) /
            predictions.shape[0])

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.



### Logistic regression L2

In [5]:
batch_size = 128
L2_reg = 0.005  # ratio for L2 regularization

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size,
                                             image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                logits=logits) +
        L2_reg * tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 8001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run([optimizer, loss, train_prediction],
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" %
                  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" %
                  accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 32.407227
Minibatch accuracy: 8.6%
Validation accuracy: 11.6%
Minibatch loss at step 500: 10.250906
Minibatch accuracy: 77.3%
Validation accuracy: 70.9%
Minibatch loss at step 1000: 6.633504
Minibatch accuracy: 75.8%
Validation accuracy: 73.7%
Minibatch loss at step 1500: 3.613878
Minibatch accuracy: 83.6%
Validation accuracy: 76.1%
Minibatch loss at step 2000: 2.487192
Minibatch accuracy: 80.5%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.760822
Minibatch accuracy: 80.5%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 1.313916
Minibatch accuracy: 82.8%
Validation accuracy: 80.3%
Minibatch loss at step 3500: 1.059580
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 4000: 0.823606
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 4500: 0.777181
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 5000: 0.651311
Minibatch accuracy: 85.2%
Validation accura

### Neural Network L2

In [16]:
n_hidden = 1024  # Number of hidden nodes
batch_size = 128
L2_reg = 0.001  # ratio for L2 regularization

g = tf.Graph()
with g.as_default():

    # Placehodlers for training dataset minibatches
    tf_train_dataset = tf.placeholder(np.float32,
                                      shape=(batch_size,
                                             image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32,
                                     shape=(batch_size, num_labels))

    # Constants for validation and test datasets
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Define Weights and biases as variables for hidden layer
    W_0 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
    b_0 = tf.Variable(tf.zeros([n_hidden]))

    # Define Weights and biases as variables for logistic regression layer
    W_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    b_1 = tf.Variable(tf.zeros([num_labels]))

    logits_0 = tf.nn.relu(tf.matmul(tf_train_dataset, W_0) +
                          b_0)  # shape will be (batch_size, n_hidden)

    logits_1 = tf.matmul(
        logits_0, W_1) + b_1  # now the shape is (batch_size, num_labels)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_1)
        + L2_reg * (tf.nn.l2_loss(W_0) + tf.nn.l2_loss(W_1)))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions
    train_prediction = tf.nn.softmax(logits_1)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W_0) + b_0), W_1) +
        b_1)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W_0) + b_0), W_1) +
        b_1)

In [8]:
n_steps = 5001  # number of steps to be taken

with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(n_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Prepare minibatch data
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                     feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch  train and validation accuracy: %.1f%%, %.1f%%" %
                  (accuracy(predictions, batch_labels),
                   accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 637.958374
Minibatch  train and validation accuracy: 10.2%, 28.6%
Minibatch loss at step 500: 195.967377
Minibatch  train and validation accuracy: 82.8%, 78.6%
Minibatch loss at step 1000: 114.814377
Minibatch  train and validation accuracy: 80.5%, 80.9%
Minibatch loss at step 1500: 67.975952
Minibatch  train and validation accuracy: 89.8%, 82.1%
Minibatch loss at step 2000: 41.184597
Minibatch  train and validation accuracy: 90.6%, 84.2%
Minibatch loss at step 2500: 25.049202
Minibatch  train and validation accuracy: 90.6%, 85.1%
Minibatch loss at step 3000: 15.451139
Minibatch  train and validation accuracy: 86.7%, 85.9%
Minibatch loss at step 3500: 9.595225
Minibatch  train and validation accuracy: 85.9%, 86.3%
Minibatch loss at step 4000: 5.985088
Minibatch  train and validation accuracy: 90.6%, 86.7%
Minibatch loss at step 4500: 3.823195
Minibatch  train and validation accuracy: 89.8%, 86.9%
Minibatch loss at step 5000: 2.534273
Minibatch  tra

### Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches.

In [17]:
n_steps = 5001 # number of steps to be taken
mod_batches = 3 # just use first a few batches over and over again


# tf.reset_default_graph()
with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialibzed')
    for step in range(n_steps):
        
        offset = (step * batch_size) % (mod_batches * batch_size - batch_size) # offset changed
        
        # Prepare minibatch data
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch  train and validation accuracy: %.1f%%, %.1f%%" 
                % (accuracy(predictions, batch_labels), 
                accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialibzed
Minibatch loss at step 0: 656.678955
Minibatch  train and validation accuracy: 9.4%, 26.8%
Minibatch loss at step 500: 190.935410
Minibatch  train and validation accuracy: 100.0%, 70.7%
Minibatch loss at step 1000: 115.793686
Minibatch  train and validation accuracy: 100.0%, 70.7%
Minibatch loss at step 1500: 70.223648
Minibatch  train and validation accuracy: 100.0%, 70.6%
Minibatch loss at step 2000: 42.587505
Minibatch  train and validation accuracy: 100.0%, 70.6%
Minibatch loss at step 2500: 25.827465
Minibatch  train and validation accuracy: 100.0%, 70.8%
Minibatch loss at step 3000: 15.663411
Minibatch  train and validation accuracy: 100.0%, 71.2%
Minibatch loss at step 3500: 9.499949
Minibatch  train and validation accuracy: 100.0%, 71.7%
Minibatch loss at step 4000: 5.763868
Minibatch  train and validation accuracy: 100.0%, 72.3%
Minibatch loss at step 4500: 3.502412
Minibatch  train and validation accuracy: 100.0%, 72.9%
Minibatch loss at step 5000: 2.136449
Minib

### Neural network with dropout applied to hidden layer

In [18]:
n_hidden = 1024  # Number of hidden nodes
batch_size = 128
L2_reg = 0.001  # ratio for L2 regularization

g = tf.Graph()
with g.as_default():

    # Placehodlers for training dataset minibatches
    tf_train_dataset = tf.placeholder(np.float32,
                                      shape=(batch_size,
                                             image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32,
                                     shape=(batch_size, num_labels))

    # Constants for validation and test datasets
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Define Weights and biases as variables for hidden layer
    W_0 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
    b_0 = tf.Variable(tf.zeros([n_hidden]))

    # Define Weights and biases as variables for logistic regression layer
    W_1 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    b_1 = tf.Variable(tf.zeros([num_labels]))

    logits_0 = tf.nn.relu(tf.matmul(tf_train_dataset, W_0) +
                          b_0)  # shape will be (batch_size, n_hidden)
    
    dropout_0 = tf.nn.dropout(logits_0, 0.5)

    logits_1 = tf.matmul(
        logits_0, W_1) + b_1  # now the shape is (batch_size, num_labels)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_1)
        + L2_reg * (tf.nn.l2_loss(W_0) + tf.nn.l2_loss(W_1)))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions
    train_prediction = tf.nn.softmax(logits_1)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W_0) + b_0), W_1) +
        b_1)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W_0) + b_0), W_1) +
        b_1)

In [19]:
n_steps = 5001 # number of steps to be taken
mod_batches = 3 # just use first a few batches over and over again


# tf.reset_default_graph()
with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialibzed')
    for step in range(n_steps):
        
        offset = (step * batch_size) % (mod_batches * batch_size - batch_size) # offset changed
        
        # Prepare minibatch data
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch  train and validation accuracy: %.1f%%, %.1f%%" 
                % (accuracy(predictions, batch_labels), 
                accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialibzed
Minibatch loss at step 0: 648.965454
Minibatch  train and validation accuracy: 7.8%, 22.4%
Minibatch loss at step 500: 190.311462
Minibatch  train and validation accuracy: 100.0%, 71.3%
Minibatch loss at step 1000: 115.415314
Minibatch  train and validation accuracy: 100.0%, 71.3%
Minibatch loss at step 1500: 69.994186
Minibatch  train and validation accuracy: 100.0%, 71.4%
Minibatch loss at step 2000: 42.448349
Minibatch  train and validation accuracy: 100.0%, 71.4%
Minibatch loss at step 2500: 25.743107
Minibatch  train and validation accuracy: 100.0%, 71.5%
Minibatch loss at step 3000: 15.612328
Minibatch  train and validation accuracy: 100.0%, 71.6%
Minibatch loss at step 3500: 9.469320
Minibatch  train and validation accuracy: 100.0%, 72.0%
Minibatch loss at step 4000: 5.745767
Minibatch  train and validation accuracy: 100.0%, 72.5%
Minibatch loss at step 4500: 3.491757
Minibatch  train and validation accuracy: 100.0%, 73.4%
Minibatch loss at step 5000: 2.130183
Minib

### To be stronger

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

- global_step = tf.Variable(0)  # count the number of steps taken.
- learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
- optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


In [20]:
n_hidden_3 = 1024  # Number of hidden nodes for hidden layer 3
n_hidden_2 = 300   # Number of hidden nodes for hidden layer 2
n_hidden_1 = 50    # Number of hidden nodes for hidden layer 1
batch_size = 128
L2_reg = 0.001 # ratio for L2 regularization

g = tf.Graph()
with g.as_default():
    
    # Placehodlers for training dataset minibatches
    tf_train_dataset = tf.placeholder(np.float32, shape=(batch_size, image_size * image_size), name='ph1')
    tf_train_labels = tf.placeholder(np.float32, shape=(batch_size, num_labels), name='ph2')
    
    # Constants for validation and test datasets
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Use learning rate decay
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 5000, 0.95, staircase=True)
    
    # Define Weights and biases as variables for hidden layers
    W_3 = tf.get_variable("W3", shape=[image_size * image_size, n_hidden_3],
                                initializer=tf.contrib.layers.xavier_initializer())
    b_3 = tf.Variable(tf.zeros([n_hidden_3]))
    
    W_2 = tf.get_variable("W2", shape=[n_hidden_3, n_hidden_2],
                                initializer=tf.contrib.layers.xavier_initializer())
    b_2 = tf.Variable(tf.zeros([n_hidden_2]))
    
    W_1 = tf.get_variable("W1", shape=[n_hidden_2, n_hidden_1],
                                initializer=tf.contrib.layers.xavier_initializer())
    b_1 = tf.Variable(tf.zeros([n_hidden_1]))
    
    # Define Weights and biases as variables for logistic regression layer
    W_0 = tf.get_variable("W0", shape=[n_hidden_1, num_labels],
                                initializer=tf.contrib.layers.xavier_initializer())
    b_0 = tf.Variable(tf.zeros([num_labels]))
    
    logits_3 = tf.nn.relu(tf.matmul(tf_train_dataset, W_3) + b_3) # shape will be (batch_size, n_hidden_3)
    dropout_3 = tf.nn.dropout(logits_3, 0.5)
    
    logits_2 = tf.nn.relu(tf.matmul(dropout_3, W_2) + b_2) # shape will be (batch_size, n_hidden_2)
    dropout_2 = tf.nn.dropout(logits_2, 0.5)
    
    logits_1 = tf.nn.relu(tf.matmul(dropout_2, W_1) + b_1) # shape will be (batch_size, n_hidden_1)
    dropout_1 = tf.nn.dropout(logits_1, 0.5)

    logits_0 = tf.matmul(dropout_1, W_0) + b_0 # now the shape is (batch_size, num_labels)
    
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_0)
            + L2_reg * (tf.nn.l2_loss(W_0) + tf.nn.l2_loss(W_1) 
                        + tf.nn.l2_loss(W_2) + tf.nn.l2_loss(W_3)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions
    def get_logits(dataset):
        get_logits_3 = tf.nn.relu(tf.matmul(dataset, W_3) + b_3)
        get_logits_2 = tf.nn.relu(tf.matmul(get_logits_3, W_2) + b_2)
        get_logits_1 = tf.nn.relu(tf.matmul(get_logits_2, W_1) + b_1)
        get_logits_0 = tf.matmul(get_logits_1, W_0) + b_0
        return get_logits_0

    train_prediction = tf.nn.softmax(get_logits(tf_train_dataset))
    valid_prediction = tf.nn.softmax(get_logits(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_logits(tf_test_dataset))

In [ ]:
n_epochs = 200  # number of epochs

# early-stopping parameters
patience = 5000  # 允许遍历的最大累计本数
patience_increase = 2  # 效果没有提升时，增加需要学习的样本数，乘以2
improvement_threshold = 0.995  # 效果提升指标
best_valid_loss = np.inf

start_time = timeit.default_timer()
n_train_batches = train_dataset.shape[0] // batch_size
valid_freq = valid_freq = min(n_train_batches, patience // 2)


with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()

    print('Initialized')
    done_looping = False
    epoch = 0
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):

            batch_data = \
                train_dataset[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            batch_labels = \
                train_labels[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]

            feed_dict = {
                tf_train_dataset: batch_data,
                tf_train_labels: batch_labels
            }
            _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                         feed_dict=feed_dict)
            
            # 全局迭代轮次
            iter = (epoch - 1) * n_train_batches + minibatch_index  
            
            # 显示输出设置
            if (iter + 1) % valid_freq == 0:
                this_valid_loss = 1. - (accuracy(valid_prediction.eval(), valid_labels) / 100.)

                print(
                    "Minibatch loss at epoch %i and iter %i: %f and learning rate: %f"
                    % (epoch, iter, l, sess.run(learning_rate)))
                print(
                    "Minibatch  train, validation accuracy: %.1f%%, %.1f%%"
                    % (accuracy(predictions, batch_labels),
                       accuracy(valid_prediction.eval(), valid_labels)))
                
                if this_valid_loss < best_valid_loss:
                    # loss要小于improvement_threshold倍的best_valid_loss，才改变允许遍历的最大样本数
                    if this_valid_loss < best_valid_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_valid_loss = this_valid_loss

                    params = (sess.run(W_0), sess.run(b_0), sess.run(W_1),
                              sess.run(b_1), sess.run(W_2), sess.run(b_2),
                              sess.run(W_3), sess.run(b_3))

                    # save the best model
                    with open('best_model_params.pkl', 'wb') as f:
                        pickle.dump(params, f)
                    print('Model saved')
            
            # 检测是否达到需要历的最大累计本数
            if patience <= iter:
                done_looping = True
                break

    print("Final Test accuracy: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))
    
    # 加载参数
    with open('best_model_params.pkl', 'rb') as f:
        W_0_best, b_0_best, W_1_best, b_1_best, W_2_best, b_2_best, W_3_best, b_3_best = pickle.load(
            f)

    W_0_init, b_0_init = tf.assign(W_0, W_0_best), tf.assign(b_0, b_0_best)
    W_1_init, b_1_init = tf.assign(W_1, W_1_best), tf.assign(b_1, b_1_best)
    W_2_init, b_2_init = tf.assign(W_2, W_2_best), tf.assign(b_2, b_2_best)
    W_3_init, b_3_init = tf.assign(W_3, W_3_best), tf.assign(b_3, b_3_best)

    sess.run([
        W_0_init, b_0_init, W_1_init, b_1_init, W_2_init, b_2_init, W_3_init,
        b_3_init
    ])

    print("Test accuracy with the best model: %.1f%%" %
          accuracy(test_prediction.eval(), test_labels))
    
    end_time = timeit.default_timer()
    print('Total run time %.4f minutes' % ((end_time - start_time) / 60.))

Initialized
Minibatch loss at epoch 1 and iter 1561: 1.418303 and learning rate: 0.050000
Minibatch  train, validation accuracy: 81.2%, 83.4%
Model saved
Minibatch loss at epoch 2 and iter 3123: 1.178277 and learning rate: 0.050000
Minibatch  train, validation accuracy: 85.9%, 84.5%
Model saved
Minibatch loss at epoch 3 and iter 4685: 1.111015 and learning rate: 0.050000
Minibatch  train, validation accuracy: 84.4%, 85.3%
Model saved
Minibatch loss at epoch 4 and iter 6247: 1.028409 and learning rate: 0.047500
Minibatch  train, validation accuracy: 85.9%, 85.9%
Model saved
Minibatch loss at epoch 5 and iter 7809: 0.948392 and learning rate: 0.047500
Minibatch  train, validation accuracy: 87.5%, 86.3%
Model saved
Minibatch loss at epoch 6 and iter 9371: 0.918184 and learning rate: 0.047500
Minibatch  train, validation accuracy: 86.7%, 86.8%
Model saved
Minibatch loss at epoch 7 and iter 10933: 0.776602 and learning rate: 0.045125
Minibatch  train, validation accuracy: 85.2%, 87.3%
Model

Minibatch loss at epoch 60 and iter 93719: 0.496211 and learning rate: 0.019861
Minibatch  train, validation accuracy: 93.0%, 90.1%
Minibatch loss at epoch 61 and iter 95281: 0.478782 and learning rate: 0.018868
Minibatch  train, validation accuracy: 92.2%, 90.1%
Minibatch loss at epoch 62 and iter 96843: 0.519758 and learning rate: 0.018868
Minibatch  train, validation accuracy: 92.2%, 90.0%
Minibatch loss at epoch 63 and iter 98405: 0.501893 and learning rate: 0.018868
Minibatch  train, validation accuracy: 91.4%, 90.1%
Minibatch loss at epoch 64 and iter 99967: 0.482345 and learning rate: 0.018868
Minibatch  train, validation accuracy: 92.2%, 90.2%
Model saved
